In [ ]:
! pip install datasets transformers seqeval

In [ ]:
 !huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git config --global credential.helper store

In [ ]:
!apt install git-lfs
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
import transformers

print(transformers.__version__)

4.15.0


In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
#bert-base-multilingual-cased
model_checkpoint = 'DeepPavlov/rubert-base-cased-sentence'


batch_size = 16

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
import json
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
f = open('gdrive/My Drive/Colab Notebooks/homographs/train.json')
data_train = json.load(f)
f.close()
#data_train = data_train[:2000]

In [ ]:
len(data_train)

140954

In [ ]:
f = open('gdrive/My Drive/Colab Notebooks/homographs/test.json')
data_test = json.load(f)
f.close()
#data_test=data_test[:4]

In [ ]:
len(data_test)

17076

In [ ]:
tag_all = set()
tag2 = []
for data in data_train:
  for w in data['source'].split():
    if w.isupper() and w not in tag_all:
       tag2.append(w)
    if w.isupper():
      tag_all.add(w)

In [ ]:
list_data_train = []
i = 0
for t in tag2:
  tmp = []
  while i < len(data_train) and t in data_train[i]['source'].split():
    tmp.append(data_train[i])
    i += 1
  list_data_train.append(tmp)

list_data_test = []
i = 0
for t in tag2:
  tmp = []
  while i < len(data_test) and t in data_test[i]['source'].split():
    tmp.append(data_test[i])
    i += 1
  list_data_test.append(tmp)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def listmerge3(lstlst):
    all=[]
    for lst in lstlst:
      all.extend(lst)
    return all


In [ ]:
data_train = listmerge3(list_data_train[1300:])
data_test = listmerge3(list_data_test[1300:])

In [ ]:
len(list_data_train)/100

14.09

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
len(data_train)

10394

In [ ]:
tokens = []
for data in data_train:
    tokens.append(data['source'].lower().split())

In [ ]:
tokens_test = []
for data in data_test:
    tokens_test.append(data['source'].lower().split())

In [ ]:
tags = set()
tags_result = set()
for data in data_train:
  idx = 0
  for w in data['source'].split():
    if w.isupper():
      tags.add("B-" + data['target'].split()[idx])
      tags_result.add(data['target'].split()[idx])

    idx += 1
tags.add('O')
tags = sorted(tags)
tags_result = sorted(tags_result)
tags_result.append('O')

In [ ]:
omo = []
for data in data_train:
  idx = 0
  s = []
  for w in data['source'].split():

    if w.isupper():
      s.append('B-' + data['target'].split()[idx])
    else:
        s.append('O')
    idx += 1
  omo.append(s)
omo[:2]

[['O',
  'O',
  'O',
  'B-сплЕтен',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'B-сплЕтен',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O']]

In [ ]:
tag2id = {tag: id for id, tag in enumerate(tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
omograph = []
for data in data_train:
  idx = 0
  s = []
  for w in data['source'].split():

    if w.isupper():
      s.append(tag2id['B-'+ data['target'].split()[idx]])
    else:
        s.append(tag2id['O'])
    idx += 1
  omograph.append(s)
omograph[:2]

[[200, 200, 200, 14, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200],
 [200, 200, 200, 14, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200]]

In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
my_dict = {'id': [str(i) for i in range(len(data_train))],
           'tokens': tokens,
            'ner_tags': omograph}

from datasets import Dataset
dataset = Dataset.from_dict(my_dict)


In [ ]:
omograph_t = []
for data in data_test:
  idx = 0
  s = []
  for w in data['source'].split():
        s.append(tag2id['O'])
  omograph_t.append(s)
omograph_t[:2]

[[200, 200, 200, 200, 200, 200, 200, 200, 200, 200],
 [200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200]]

In [ ]:
from datasets import Dataset
dataset = Dataset.from_dict(my_dict)

my_dict = {'id': [str(i) for i in range(int(len(data_train) * 0.3))],
           'tokens': tokens[:int(len(data_train) * 0.3)],
            'ner_tags': omograph[:int(len(data_train) * 0.3)]}

dataset2 = Dataset.from_dict(my_dict)

my_dict = {'id': [str(i) for i in range(len(data_test))],
           'tokens': tokens_test,
            'ner_tags': omograph_t

          }

dataset3 = Dataset.from_dict(my_dict)

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
from datasets import DatasetDict
datasets=DatasetDict({'train': dataset,
                      'validation':dataset2,
                      'test':dataset3})
datasets


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 10394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3118
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1260
    })
})

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[101, 845, 24788, 16542, 55374, 1386, 612, 3835, 128, 1997, 18462, 16324, 12352, 2058, 130, 3998, 18878, 2058, 11932, 36989, 102], [101, 16547, 35057, 9502, 103122, 35043, 55374, 1386, 128, 37864, 128, 11765, 128, 26959, 104988, 70480, 1703, 86371, 626, 90629, 102], [101, 5539, 60983, 54662, 52324, 78539, 1502, 19099, 55374, 1386, 128, 22189, 70411, 39384, 32608, 128, 4427, 25997, 26853, 7470, 108113, 19904, 1758, 14178, 72508, 3565, 102], [101, 16570, 15514, 20883, 47940, 55374, 1386, 851, 28021, 845, 42110, 22914, 9643, 851, 845, 63757, 102], [101, 5539, 45078, 9450, 24429, 9529, 112993, 130, 11086, 94151, 55374, 1386, 128, 67783, 12355, 119525, 9210, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
label_list = tags

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
model_name = model_checkpoint.split("/")[-1]
batch_size=16
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=11,
    save_steps=6000, # after # steps model is saved
    weight_decay=0.01,
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}",
)
args.device


/usr/local/lib/python3.7/dist-packages/transformers/training_args.py:926: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case Ann2020/rubert-base-cased-sentence-finetuned-ner).
  FutureWarning,


device(type='cuda', index=0)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")

In [ ]:
labels = [label_list[i] for i in datasets['train'][0]['ner_tags']]
metric.compute(predictions=[labels], references=[labels])

{'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'сплЕтен': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0}}

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
torch.cuda.empty_cache()


In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],

    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/test-ner is already a clone of https://huggingface.co/Ann2020/rubert-base-cased-sentence-finetuned-ner. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, id, ner_tags.
***** Running training *****
  Num examples = 10394
  Num Epochs = 11
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7150


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.325400,0.054198,0.828242,0.838814,0.833495,0.988868
2,0.063600,0.033349,0.850272,0.845305,0.847781,0.990025
3,0.044500,0.019854,0.916811,0.918001,0.917405,0.994400
4,0.025200,0.014431,0.942900,0.953916,0.948376,0.996339
5,0.018500,0.011063,0.949465,0.959325,0.954369,0.996769
6,0.016200,0.009176,0.958387,0.966681,0.962516,0.997348
7,0.011100,0.006870,0.969625,0.973821,0.971718,0.997954
8,0.009400,0.006283,0.972503,0.979446,0.975962,0.998263
9,0.008600,0.004755,0.979914,0.981610,0.980761,0.998667
10,0.007200,0.004111,0.981626,0.982475,0.982050,0.998735


The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, id, ner_tags.
***** Running Evaluation *****
  Num examples = 3118
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, id, ner_tags.
***** Running

TrainOutput(global_step=7150, training_loss=0.04147883955415312, metrics={'train_runtime': 3608.1602, 'train_samples_per_second': 31.688, 'train_steps_per_second': 1.982, 'total_flos': 3690852509265948.0, 'train_loss': 0.04147883955415312, 'epoch': 11.0})

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, id, ner_tags.
***** Running Prediction *****
  Num examples = 3118
  Batch size = 16


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'overall_accuracy': 0.9987750212015561,
 'overall_f1': 0.9828126689006593,
 'overall_precision': 0.9820695614603586,
 'overall_recall': 0.9835569017741238,
 'сУку': {'f1': 0.0, 'number': 0, 'precision': 0.0, 'recall': 0.0},
 'спОрите': {'f1': 1.0, 'number': 38, 'precision': 1.0, 'recall': 1.0},
 'сплЕтен': {'f1': 1.0, 'number': 200, 'precision': 1.0, 'recall': 1.0},
 'сплетЕн': {'f1': 1.0, 'number': 10, 'precision': 1.0, 'recall': 1.0},
 'спорИте': {'f1': 1.0, 'number': 36, 'precision': 1.0, 'recall': 1.0},
 'спрОсите': {'f1': 0.7999999999999999,
  'number': 16,
  'precision': 0.8571428571428571,
  'recall': 0.75},
 'спросИте': {'f1': 0.9871244635193134,
  'number': 232,
  'precision': 0.9829059829059829,
  'recall': 0.9913793103448276},
 'срЕдам': {'f1': 0.9795918367346939,
  'number': 96,
  'precision': 0.96,
  'recall': 1.0},
 'срЕдах': {'f1': 0.9375,
  'number': 90,
  'precision': 0.8823529411764706,
  'recall': 1.0},
 'срЕду': {'f1': 0.9937888198757764,
  'number': 81,
  'precisi

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
predictions

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, id, ner_tags.
***** Running Prediction *****
  Num examples = 3118
  Batch size = 16


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[ 14, 200, 200, ...,   0,   0,   0],
       [ 14, 200, 200, ...,   0,   0,   0],
       [ 14, 200, 200, ...,   0,   0,   0],
       ...,
       [ 90, 200,  90, ...,   0,   0,   0],
       [ 90, 200, 200, ...,   0,   0,   0],
       [ 90, 200, 200, ...,   0,   0,   0]])

In [ ]:
labels


array([[-100,  200,  200, ..., -100, -100, -100],
       [-100,  200,  200, ..., -100, -100, -100],
       [-100,  200,  200, ..., -100, -100, -100],
       ...,
       [-100,  200,   90, ..., -100, -100, -100],
       [-100,  200,  200, ..., -100, -100, -100],
       [-100,  200,  200, ..., -100, -100, -100]])

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
tokenized_datasets["test"]

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, id, ner_tags.
***** Running Prediction *****
  Num examples = 1260
  Batch size = 16


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Dataset({
    features: ['attention_mask', 'id', 'input_ids', 'labels', 'ner_tags', 'token_type_ids', 'tokens'],
    num_rows: 1260
})

In [ ]:
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [tags_result[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_predictions

[['O', 'O', 'сплЕтен', 'сплЕтен', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'сплЕтен',
  'сплЕтен',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'сплЕтен',
  'сплЕтен',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'сплЕтен',
  'сплЕтен',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'сплЕтен',
  'сплЕтен'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'сплЕтен',
  'сплЕтен',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'сплЕтен',
  'сплЕтен',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O'

In [ ]:
len(true_predictions)

1260

In [ ]:
res_test = []
k=0
for i in range(len(data_test)):
  data = data_test[i]['source'].split()
  ind = -1
  flag = True
  for j in range(len(data)):
    if data[j].isupper():
      ind = j
      break
  for el in set(true_predictions[i]):
    if el.lower() == data[ind].lower():
      res_test.append(el)
      flag = False
      break
  if flag:
    res_test.append(res_test[-1])
    k+=1
k



10

In [ ]:
fout=open('output1100_1200.txt', 'w')
with fout as f:
  for el in res_test:
    print(el, file=f)

In [ ]:
data_test[:20]

[{'source': 'чем больше СПЛЕТЕН и грязи , тем лучше для него'},
 {'source': 'если же вы сами стали предметом СПЛЕТЕН - наяву вас ожидает приятный сюрприз'},
 {'source': 'в связи с этим большое количество слухов , СПЛЕТЕН и домыслов заполонили соцсети'},
 {'source': 'вероятнее всего , все начнется со слухов , злых СПЛЕТЕН , способных погубить вашу репутацию'},
 {'source': 'грязь - символ богатства , бурной деятельности , обилия событий , СПЛЕТЕН'},
 {'source': 'если во сне убили любимого человека , то остерегайтесь злых разговоров и СПЛЕТЕН вокруг вас и близких родственников'},
 {'source': '- в настоящее время анастасия заворотнюк , ее жизнь и здоровье стали объектом для СПЛЕТЕН , откровенной неправды , передергивания фактов и спекуляции на имени артистки'},
 {'source': 'минерал наделяют магическими свойствами и используют для защиты от сглаза , порчи , проклятий , СПЛЕТЕН и зависти'},
 {'source': 'злословие - , в еврейской и библейской религиозной традиции запрет произнесения любых сло